In [39]:
# This is a cell to hide code snippets from displaying
# This must be at first cell!

from IPython.display import HTML

hide_me = ''
HTML('''<script>
code_show=true; 
function code_toggle() {
  if (code_show) {
    $('div.input').each(function(id) {
      el = $(this).find('.cm-variable:first');
      if (id == 0 || el.text() == 'hide_me') {
        $(this).hide();
      }
    });
    $('div.output_prompt').css('opacity', 0);
  } else {
    $('div.input').each(function(id) {
      $(this).show();
    });
    $('div.output_prompt').css('opacity', 1);
  }
  code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>

<form action="javascript:code_toggle()"><input style="opacity:0" type="submit" value="Click here to toggle on/off the raw code."></form>''')

<div style="text-align: center;">
    <h1> Neutron star crust cooling: The peculiar case of Terzan 5 X-2</h1>
    <h4>Marcella J.P. Wijngaarden, Laura Ootes, Rudy Wijnands, Dany Page <br><br>
    NAC 22-25 May 2017</h4>
</div>
<font color='purple'>This notebook contains support material for the poster presented at the <a                href="http://www.astronomenclub.nl/conferentie/">NAC 2017 </a>.</font> 

<strong>What is neutron star crust cooling and what can we learn from observing it?</strong>

<strong>What can you find in this notebook?</strong>
This notebook contains the following sections:
<ol>
<a href="#coolingdata"><li>The cooling data</li></a>
    <li><a href="#modelimp">Model 1: Unusual crust properties</a> 
        <ol><li><a href="#movie1">Movie of evolution of internal temperature profile</a></li></ol>
    </li>
<a href="#references"><li>Further references</li></a>
</ol>

<h3 id="coolingdata">1. The cooling data </h3>

We compare our models to the cooling data analyzed by <a href="#references">Degenaar et al. (2013)</a> and <a href="#references">Degenaar et al (2015)</a> and describe here in short how the data was analyzed. Eight Terzan 5 X-2 X-ray spectra were obtained in quiescence with <i>Chandra</i>/ACIS between September 2011 and July 2014. Two spectra were obtained (in 2003 and 2009) before the accretion outburst. These can be used to study the thermal evolution after the accretion outburst in 2010. All spectral data sets were fitted simultaneously with the neutron star atmosphere model NSATMOS of Heinke et al. (2006a). In the model the mass and radius were kept fixed at M = 1.4 M$_{\odot}$ and R = 10 km, the distance at D = 5.5 kpc and the normalization at unity (implying that the whole neutron star is radiating). This leaves the only free fit parameter as the neutron star effective temperature $kT$. See the table below for a summary of the obtained temperatures.

\begin{array}{c c c c c}
    \hline \hline
    \text{Epoch} & \text{MJD} & kT_{eff}^{\infty} & F_{bol} & L_{bol} \\
      &  & (eV) & (10^{-13} erg ~cm^{-2} ~s^{-1}) & (10^{32} erg ~s^{-1}) \\
    \hline
    2003/2009 & 52833.5/55027.5 & 73.6 \pm 1.6 & 1.78 \pm 0.20 & 6.44 \pm 0.72 \\
    2011 ~\text{Feb} & 55609 			& 99.7 \pm 1.6 & 6.02 \pm 0.41 & 21.8 \pm 1.5 \\
    2011 ~\text{Apr} & 55680.5 			& 91.5 \pm 1.5 & 4.26 \pm 0.29 & 15.4 \pm 1.0 \\
    2011 ~\text{Sep} & 55810.5 			& 89.2 \pm 1.5 & 3.86 \pm 0.25 & 14.0 \pm 0.9 \\
    2012 ~\text{May} & 56060 			& 84.8 \pm 1.5 & 3.15 \pm 0.23 & 11.4 \pm 0.8 \\
    2012 ~\text{Sep} & 56187.5 			& 88.5 \pm 1.9 & 3.73 \pm 0.32 & 13.5 \pm 1.2 \\
    2012 ~\text{Oct} & 56228 			& 84.6 \pm 2.0 & 3.11 \pm 0.32 & 11.3 \pm 1.2 \\
    2013 ~\text{Feb} & 56340		 	& 82.8 \pm 1.2 & 2.86 \pm 0.17 & 10.4 \pm 1.2 \\
    2014 ~\text{July}& 56853		 	& 82.7 \pm 1.2 & 2.86 \pm 0.17 & 10.4 \pm 1.2 \\
    \hline
    \hline
\end{array}


<a id="modelimp"></a><h3>2. Model 1: Unusual crust properties</h3>

<a id="movie1"></a><strong>Movie</strong>

This movie shows how the calculated internal temperature structure of the neutron star crust evolves during the outburst and the subsequent cooling. The layer above the neutron drip density is highlighed as well as the pasta layer just before the boundary with the neutron star core. Due to the large amount of impurities in the neutron drip layer it has a significantly lower thermal conductivity than the lower density regime, this is clearly visible in the movie in the timescales of the heat propagation.

In [2]:
hide_me

%run 'anim_to_html.ipynb'
# %pylab notebook
%matplotlib inline
import NSCool_Py
import Boundary_Py
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import rc
import scipy.optimize as optimize
import scipy.interpolate
import os

# For inline interactive plot:
# %matplotlib notebook
fsize=15

# Read crust cooling temperature data
import Read_Data
data_time, data_Teff, data_dT1 , data_dT2 = Read_Data.Terzan5_X2() # KS_1731_260()
data_Ltime = np.log(data_time)/np.log(10.)

# print data_time
# print data_Teff

# Constants
eV    = 1.16044e4
day   = 86400.
year  = 365.25 * 864002
t_pre_outburst = 20 * 365.25

# Use this filename for regular use
# filename0  = 'Cool_APR-1.4_Acc-Dany-08_Choose-0-0_Mdot=0.2Edd_7.5_Termo=200keV_aTD=0_Q3zones.in'
# Use this filename for fit use
filename0 = 'Cool_Accretion_TerzanX2.in'
directory0 = 'Ter5_X-2'
directory = 'Models/' + directory0
filename = directory + '/' + filename0          # Control file
filefit = directory + '/fit_input.dat'          # Name of fit input file with initial parameter guesses
                                                # these parameters + fit control can also be overwritten from the notebook.
                                                # See below for an example.
# print filename
# print filefit


MODE = 'PYTO'
ip_max = 1
chi2 = 0.

# Start NSCool
NSCool_Py.nscool_start(filename,filename0,directory,directory0,MODE,ip_max,chi2)

# Initialize fit and read in fit input
NSCool_Py.nscool_fit_init(filename,filename0,directory,directory0,MODE,filefit,0.0)

ns_mass = NSCool_Py.fit_param.emas_new_d
ns_radius = NSCool_Py.fit_param.rad_new_d
ns_Tcore = 6.7e+07   # NSCool_Py.fit_param.temp_core
ns_yenv = 6.6e+07
ns_Qimp1 = 150 # 250.0 #1.0
ns_Qimp2 = 1
NSCool_Py.impurity_t.q_imp_t[3] = 1.0

# These are the best fit values for bolometric correction of 1.8
# ns_qshallow = 3392.20 
# ns_rhoshallow = 1.5788943360e+10 

# These are the best fit values for bolometric correction of 2.0
ns_qshallow =  3019.00 
ns_rhoshallow = 1.5721232384e+10 
    
NSCool_Py.nscool_function(ns_yenv,ns_radius,ns_mass,ns_Tcore,ns_qshallow,ns_rhoshallow,ns_Qimp1,ns_Qimp2)

# Time indices take values from 0 to jmax-1:
jmax = NSCool_Py.results.istep_max

# 1D time dependent arrays:
# j takes jmax values: 0, ..., jmax-1
Age0  = NSCool_Py.accretion.t_acc0 + NSCool_Py.accretion.t_acc2[0]
Age0  = Age0/day

Age   = NSCool_Py.results.age[0:jmax]/day
LAge  = np.log(Age)/np.log(10.)
Teff  = NSCool_Py.results.teff[0:jmax]/eV

# print  Teff[50],Teff[52], Teff[60]
# print Age[50]- t_pre_outburst, Age[52]- t_pre_outburst, Age[60]- t_pre_outburst

# Age1 and Teff1 is the data after outburst
Teff1 = Teff[np.where(Age > Age0)]
Age1 = Age[np.where(Age > Age0)] - Age0


# # Plot cooling curve after the outburst has ended (log x-scale)
# Read_Data.plot_NSCool(Age1, Teff1, data_time=data_time, data_teff=data_Teff, yerr=[data_dT1, data_dT2],
#                       x_lim=[0.1, 1e5], xlog=True, title='Terzan 5 X-2 Cooling after outburst')


# # Plot full outburst with subsequent cooling (linear x-scale)
# Read_Data.plot_NSCool(Age - t_pre_outburst, Teff, data_time=(data_time+Age0-t_pre_outburst), 
#                       x_lim=[-1000, 4000], data_teff=data_Teff, yerr=[data_dT1, data_dT2],
#                       title='Terzan 5 X-2 full outburst curve')
# print '-------------------------------'
# print("{0: <13} {1: <5} {2:.2f} ".format('Chi^2 ', '=', float(NSCool_Py.iterations.pychi2))) 

In [38]:
hide_me

import warnings
warnings.filterwarnings('ignore')

matplotlib.rc('font',**{'family':'serif','sans-serif':['Avant Garde'],'size': 12})
movie_fsize=12

def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))

# Read in movie data
movie_files = [] 
for file in os.listdir('./TMP_Movie/'):
    if file.endswith('.dat'):
        movie_files.append(file)

data = np.genfromtxt('./TMP_Movie/'+movie_files[0])
meltrho = data[:,10]
meltT= np.log(data[:,-3][0])
# print meltT

# Setup the figure layout for movie
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9.5,5))

# AX1
ax1.set_xlabel('Days since beginning outburst', fontsize=movie_fsize)
ax1.set_ylabel('k$_B$T${_{e}}^{\infty}$ (eV)', fontsize=movie_fsize)
ax1.plot(Age - t_pre_outburst, Teff)
# ax1.plot(current_time, current_temp, marker='o', markersize=10)
ax1.set_xlim(-50,5000)
ax1.set_title('Terzan 5 X-2 cooling curve')

# Intialize drawn objects
point, = ax1.plot([], [], marker='v', markersize=10., c='r')

# Ax2
ax2.set_xscale('log')
# ax2.set_yscale('log')
ax2.set_xlim(10**8.,10**15.)
y_min, y_max = 7.75, 8.4
ax2.set_ylim(y_min, y_max)

ax2.set_xlabel('Density (g cm$^{-3}$)', fontsize=movie_fsize)
ax2.set_ylabel('log(T) (K)', fontsize=movie_fsize)

ax2.set_title('Internal temperature profile')
# ax2.axvline(x=4e11, ls='--')
# ax2.axvline(x=8e13, ls='--')


patch = matplotlib.patches.Rectangle((4e11, y_min), 8e13, y_max, alpha=0.2, color='dodgerblue')
currentAxis = plt.gca()
currentAxis.add_patch(patch)

patch = matplotlib.patches.Rectangle((8e13, y_min), 1e15, y_max, alpha=0.2, color='darkorange')
currentAxis = plt.gca()
currentAxis.add_patch(patch)

# ax2.axvline(x=1e15, ls='--')

# Intialize drawn objects
line, = ax2.plot([], [], lw=1)

time = ax2.text(11**8, 8.35, 'time = -1 yr', fontsize=movie_fsize)
ax2.text(5e12, 8.31, 'neutron drip\nlayer', fontsize=9, ha='center')
ax2.text(3e14, 8.31, 'pasta\nlayer', fontsize=9, ha='center')

# Setup the first frame (shown before play)
def init():
    data = np.genfromtxt('./TMP_Movie/'+movie_files[0])
    rho = 10**(data[:,5])
    T = (data[:, 4])
   
    current_time = data[:,0][0]*365.25 + abs(t_pre_outburst - Age0)
    current_temp = data[:,1][0]
    
    LogTc_n_pg= 10**data[:,-3]
    LogTc_p_pg = 10**data[:,-2]
    LogT_M_pg = 10**data[:,-2]
    
    point.set_data(current_time, current_temp)
    
    line.set_data(rho, T)
    time.set_text('time = -1 yr')
    return line, time, point

# Animation function is called sequentially
def animate(i):

    data = np.genfromtxt('./TMP_Movie/'+movie_files[i])
    rho = 10**(data[:,5])
    T = (data[:, 4])
    t = data[:, 0][0]
    current_time = data[:,0][0]*365.25 + abs(t_pre_outburst - Age0)
    current_temp = data[:,1][0]

    point.set_data(current_time, current_temp)   
    line.set_data(rho, T)   
    time.set_text('t = '+'{0:.3f}'.format(t)+' yr')
    
    return line, time, point

# Call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(movie_files), interval=20, blit=True)

display_animation(anim)
# HTML(anim.to_html5_video())
 

<a id="references"></a><h3>Further references</h3>

1) Degenaar, N., Wijnands, R., Bahramian, A., et al., <b>2015</b>, <i>Neutron star crust cooling in the Terzan 5 X-ray transient swift J174805.3-244637</i>, MNRAS, 451(2), 2071–2081. https://doi.org/10.1093/mnras/stv1054

2) Degenaar, N., Wijnands, R., Brown, <b>2013</b>, <i>Continued neutron star crust cooling of the 11 Hz X-ray pulsar in Terzan 5: A challenge to heating and cooling models?</i>, The Astrophysical Journal, 775(1), 48. https://doi.org/10.1088/0004-637X/775/1/48